In [3]:
import pandas as pd
import numpy as np
import altair as alt
import eco_style
import json
alt.themes.enable('report')
import geopandas as gpd
import json
import os
import requests

charts_path = "/Users/finn/Documents/GitHub/GrowthDiagnostics/charts"

# GDP Data (to calc ratios)

In [18]:
def parse_quarters(df):
    df['quarter'] = pd.to_numeric(df.quarter.str.replace('Q', ''))
    df['month'] = (df.quarter-1)*3+1
    df['date'] = pd.to_datetime(df.year.astype(str) + '-' + df.month.astype(str) + '-01')
    return df

def parse_years(df):
    df['date'] = pd.to_datetime(df.year.astype(str) + '-01-01')
    return df

def clean_ons_series(dataset_id, series_id, date_parser, key):
    url = f'https://api.ons.gov.uk/timeseries/{series_id}/dataset/{dataset_id}/data'
    req = requests.get(url)
    data = req.json()
    data = data[key]
    df = pd.DataFrame(data)
    df = date_parser(df)
    df = df[['date', 'value']]
    df = df.dropna()
    return df

gdp_sa_cp_ids = {
    # Gross Domestic Product: chained volume measures: Seasonally adjusted £m
    # from Quarterly National Accounts
    "dataset_id" : "qna",
    "series_id" : "YBHA",
}

gdp_sa_cp = clean_ons_series(**gdp_sa_cp_ids, date_parser=parse_years, key='years')
gdp_sa_cp['value'] = pd.to_numeric(gdp_sa_cp.value)

# Exports and Imports

In [19]:
exports_sa_ids = {
    # Trade in Goods (T): WW: Exports: BOP: CP: SA
    # from Balance of Payments
    "dataset_id" : "MRET",
    "series_id" : "BOKG",
}

imports_sa_ids = {
    # Trade in Goods (T): WW: Imports: BOP: CP: SA
    # from Balance of Payments
    "dataset_id" : "MRET",
    "series_id" : "BOKH",
}

exports_sa = clean_ons_series(**exports_sa_ids, date_parser=parse_years, key='years')
exports_sa['value'] = pd.to_numeric(exports_sa.value)
imports_sa = clean_ons_series(**imports_sa_ids, date_parser=parse_years, key='years')
imports_sa['value'] = pd.to_numeric(imports_sa.value)

exports_sa = exports_sa.merge(gdp_sa_cp, on='date', how='left')
exports_sa['value'] = exports_sa.value_x / exports_sa.value_y
exports_sa['series'] = 'Exports'
exports_sa = exports_sa[['date', 'value', 'series']]

imports_sa = imports_sa.merge(gdp_sa_cp, on='date', how='left')
imports_sa['value'] = imports_sa.value_x / imports_sa.value_y
imports_sa['series'] = 'Imports'
imports_sa = imports_sa[['date', 'value', 'series']]

df = pd.concat([exports_sa, imports_sa])



In [20]:
temp_df = df.pivot_table(index='date', columns='series', values='value')
temp_df['total_trade'] = temp_df['Exports'] + temp_df['Imports']
temp_df

series,Exports,Imports,total_trade
date,,,
1997-01-01,0.177812,0.194595,0.372407
1998-01-01,0.164022,0.190129,0.354151
1999-01-01,0.159447,0.191459,0.350906
2000-01-01,0.169724,0.205895,0.375619
2001-01-01,0.166370,0.208801,0.375170
2002-01-01,0.157338,0.203121,0.360459
2003-01-01,0.149311,0.194843,0.344154
2004-01-01,0.144455,0.195377,0.339832
2005-01-01,0.151917,0.204854,0.356771
